# Pakiet numpy II

**Zadanie 1**

Proszę przygotować rozwiązanie, które pomoże obliczyć wyniki testów przeprowadzonych wśród uczniów. Wyniki mają być zapisywane w pliku _*.txt_, jego struktura wygląda następnie *nr_ucznia,pyt1,pyt_2,...,pyt_n*. Plik *answer.tx*t zawiera przykładowe wyniki testów przeprowadzonych wśrod $124$ uczniów, proszę wykoań następujące obliczenia:
1. Podaj wyniki: sumę poprawnych odpowiedzi dla każdego testu, średnią i medianę wszystich testów.
2. Podaj odchylenie standardowe i współczynnik zmieności (zapoznaj się z definicją).
3. Policz ilu ucznów uzyskało wyniki w następujących przedziałach: $0\% - 24\%$, $25\% - 49\%$, $50\% - 74\%$, $75\% - 100\%$
4. Proszę podać statystykę popawnie udzielonych odpowiedzi na pytania w testach, wartość proszę wyrazić w procentach ($\%$).

In [ ]:

import numpy as np

def analyze_test_results(filename):
  try:
    with open(filename, 'r') as f:
      lines = f.readlines()
  except FileNotFoundError:
    print(f"Plik {filename} nie został znaleziony.")
    return None

  results = []
  for line in lines:
    data = line.strip().split(',')
    if len(data) > 1:
      results.append([int(x) for x in data[1:]])

  if not results:
    print("Brak danych w pliku.")
    return None

  results_array = np.array(results)

  sum_correct = np.sum(results_array, axis=0)
  average_score = np.mean(np.sum(results_array, axis=1))
  median_score = np.median(np.sum(results_array, axis=1))

  std_deviation = np.std(np.sum(results_array, axis=1))
  variation_coefficient = std_deviation / average_score

  total_questions = results_array.shape[1]
  score_percentages = (np.sum(results_array, axis=1) / total_questions) * 100
  ranges = [
      (0, 24), (25, 49), (50, 74), (75, 100)
  ]
  range_counts = [
      np.sum((score_percentages >= start) & (score_percentages <= end))
      for start, end in ranges
  ]

  correct_answers_percentage = (np.sum(results_array, axis=0) / len(results_array)) * 100

  statistics = {
      "suma_poprawnych_odpowiedzi": sum_correct,
      "srednia_wynikow": average_score,
      "mediana_wynikow": median_score,
      "odchylenie_standardowe": std_deviation,
      "wspolczynnik_zmiennosci": variation_coefficient,
      "liczba_uczniow_w_przedzialach": dict(zip([str(r) for r in ranges], range_counts)),
      "statystyka_poprawnych_odpowiedzi_na_pytania": correct_answers_percentage,
  }

  return statistics

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

results = analyze_test_results(filename)
if results:
  print("Statystyki testów:")
  for key, value in results.items():
    print(f"{key}: {value}")

Saving answers.txt to answers (1).txt
Statystyki testów:
suma_poprawnych_odpowiedzi: [61 78 64 64 65 54 64 64 51 73 64 57 68 75]
srednia_wynikow: 7.274193548387097
mediana_wynikow: 7.0
odchylenie_standardowe: 2.163996860938556
wspolczynnik_zmiennosci: 0.29748959063900327
liczba_uczniow_w_przedzialach: {'(0, 24)': 4, '(25, 49)': 45, '(50, 74)': 68, '(75, 100)': 7}
statystyka_poprawnych_odpowiedzi_na_pytania: [49.19354839 62.90322581 51.61290323 51.61290323 52.41935484 43.5483871
 51.61290323 51.61290323 41.12903226 58.87096774 51.61290323 45.96774194
 54.83870968 60.48387097]


**Zadanie 2** (symulacja przepływu ciepła w materiałach)

Wyobraź sobie że masz siatkę 2D (dwu wymiarową tablice) reprezentującą temperatury na powierzchni prostokątnego materiału. Każda komórka siatki zawiera temperaturę w danym punkcie materiału. Celem zadania jest symulacja przepływu ciepła w materiale, korzystając z równania przewodnictwa cieplnego.

Równanie przewodnictwa cieplnego w dwóch wymiarach może być uproszczone i przedstawione jako iteracyjny wzór:
$$ T^{(n+1)}_{i,j} = \frac{1}{4} (T^{(n)}_{i+1,j} + T^{(n)}_{i-1,j} + T^{(n)}_{i,j+1} + T^{(n)}_{i,j-1}) $$

Oznacza to, że temperatura w punkcie $T_{i,j}$ w kolejnym kroku czasu $n+1$ jest średnią z temperatury sąsiednich punktów w bieżącym kroku $n$. Brzegi siatki mają stałą temperaturę (warunki brzegowe).

1. Zaimplementuj model symulacji przepływu ciepła na siatce 2D o rozmiarze $30 \times 30$. Ustaw stałe wartości temperaturowe na górnym brzegu wynoszącą $100^{o}C$, a na dolnym brzegu: $0^{o}C$. Stwórz symulację dla $50$ kroków, oblicz średnią temperaturę dla powierzchni po $25$ i $50$ krokach symulacji.
2. Zwiększ macierz do rozmiarów $100 \times 100$ i oblicz średnią temperaturę powierzchni po $25$ i $50$ krokach symulacji.
3. Ustaw stałą temperaturę dla całej powierzchni np. $20^{o}C$. Ustaw nowy punkt cieplny, którym będzie macierz $3 \times 3$ o zadanej temeperturze. Lewy górny róg tej macierzy to suma trzech ostatnich numerów numeru Twojego abumu. Np. numer indeksu $012345$, zatem punkt początkowy to $3+4+5=12$ ($12$-ty wiersz i $12$-ta kolumna). Oblicz średnią temperaturę co $20$ wiersz i $20$ kolumnę dla sitki reprezentującej powierzchnię, po zakończeniu symulacji.

In [2]:
import numpy as np
def heat_simulation(size, steps, top_temp=100, bottom_temp=0):
  grid = np.zeros((size, size))
  grid[0, :] = top_temp
  grid[-1, :] = bottom_temp

  for _ in range(steps):
    new_grid = grid.copy()
    for i in range(1, size - 1):
      for j in range(1, size - 1):
        new_grid[i, j] = 0.25 * (
            grid[i + 1, j] + grid[i - 1, j] + grid[i, j + 1] + grid[i, j - 1]
        )
    grid = new_grid

  return grid


grid_30 = heat_simulation(30, 50)
avg_temp_25_30 = np.mean(heat_simulation(30, 25))
avg_temp_50_30 = np.mean(grid_30)
print(
    "Średnia temperatura dla siatki 30x30 po 25 krokach:"
    f" {avg_temp_25_30:.2f}"
)
print(
    "Średnia temperatura dla siatki 30x30 po 50 krokach:"
    f" {avg_temp_50_30:.2f}"
)

grid_100 = heat_simulation(100, 50)
avg_temp_25_100 = np.mean(heat_simulation(100, 25))
avg_temp_50_100 = np.mean(grid_100)
print(
    "Średnia temperatura dla siatki 100x100 po 25 krokach:"
    f" {avg_temp_25_100:.2f}"
)
print(
    "Średnia temperatura dla siatki 100x100 po 50 krokach:"
    f" {avg_temp_50_100:.2f}"
)

size = 100
grid = np.full((size, size), 20)
hotspot_temp = 100


row_start = 12
col_start = 12

for i in range(row_start, min(row_start + 3, size)):
  for j in range(col_start, min(col_start + 3, size)):
    grid[i, j] = hotspot_temp


for _ in range(50):
  new_grid = grid.copy()
  for i in range(1, size - 1):
    for j in range(1, size - 1):
      new_grid[i, j] = 0.25 * (
          grid[i + 1, j] + grid[i - 1, j] + grid[i, j + 1] + grid[i, j - 1]
      )
  grid = new_grid

avg_temps = []
for row in range(0, size, 20):
  for col in range(0, size, 20):
    avg_temps.append(np.mean(grid[row : row + 20, col : col + 20]))

print("Średnie temperatury co 20 wierszy i 20 kolumn:")
print(avg_temps)

Średnia temperatura dla siatki 30x30 po 25 krokach: 10.07
Średnia temperatura dla siatki 30x30 po 50 krokach: 12.91
Średnia temperatura dla siatki 100x100 po 25 krokach: 3.26
Średnia temperatura dla siatki 100x100 po 50 krokach: 4.33
Średnie temperatury co 20 wierszy i 20 kolumn:
[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0]


**Zadanie 3** (analiza dnych pogodowych)

Stwórz fikcyjne dane pogodowe (ale realistyczne), które były zbierane przez trzy stacje pogodowe. Każda stacja powinna być zapisana w osobnej macierzy,
dane zbierane przez stację to: temperaturę ($^{o}C$), wilgotność ($\%$) oraz opady ($mm$). Stacje zbierały dane przez $7$ dni (wiersze) $4$ razy dziennie (kolumny).
1. Oblicz średnią temperaturę, wilgotność, opady dla każdego dnia w każdej stacji.
2. Znajdź dzień w którym i na której stacji zanotowano najwyższą temperaturę oraz najniższe opady.
3. Sprawdź czy były dni z ekstremalnymi warunkami tj.:
   * temperatura powyżej $30^{o}C$;
   * wilgotność powyżej $45\%$;
   * opady powyżej $20 mm$.
4. Sprawdź czy istnieją korelacje między temperaturą a wilgotnością na wszystkich stacjach pomiarowych (w tym celu zapoznaj się z funkcją `corrcoef()` z modułu `numpy`).


In [ ]:

np.random.seed(42)

station1 = np.random.randint(
    low=[15, 50, 0], high=[30, 80, 15], size=(7, 4, 3)
)
station2 = np.random.randint(
    low=[10, 40, 0], high=[25, 70, 10], size=(7, 4, 3)
)
station3 = np.random.randint(
    low=[18, 60, 5], high=[35, 90, 20], size=(7, 4, 3)
)

avg_station1 = np.mean(station1, axis=1)
avg_station2 = np.mean(station2, axis=1)
avg_station3 = np.mean(station3, axis=1)

print("Średnie wartości dla stacji 1:")
print(avg_station1)
print("Średnie wartości dla stacji 2:")
print(avg_station2)
print("Średnie wartości dla stacji 3:")
print(avg_station3)

temps = np.concatenate((station1[..., 0], station2[..., 0], station3[..., 0]))
precipitations = np.concatenate((station1[..., 2], station2[..., 2], station3[..., 2]))

max_temp_index = np.argmax(temps)
min_precipitation_index = np.argmin(precipitations)

max_temp_station = None
max_temp_day = None

if max_temp_index < station1.size:
  max_temp_station = 1
  day_index, _, _ = np.unravel_index(max_temp_index, station1.shape)
  max_temp_day = day_index
elif max_temp_index < station1.size + station2.size:
  max_temp_station = 2
  day_index, _, _ = np.unravel_index(
      max_temp_index - station1.size, station2.shape
  )
  max_temp_day = day_index
else:
  max_temp_station = 3
  day_index, _, _ = np.unravel_index(
      max_temp_index - station1.size - station2.size, station3.shape
  )
  max_temp_day = day_index

print(
    "Najwyższa temperatura zanotowana na stacji",
    max_temp_station,
    "w dniu",
    max_temp_day,
)


min_precipitation_station = None
min_precipitation_day = None

if min_precipitation_index < station1.size:
  min_precipitation_station = 1
  day_index, _, _ = np.unravel_index(min_precipitation_index, station1.shape)
  min_precipitation_day = day_index
elif min_precipitation_index < station1.size + station2.size:
  min_precipitation_station = 2
  day_index, _, _ = np.unravel_index(
      min_precipitation_index - station1.size, station2.shape
  )
  min_precipitation_day = day_index
else:
  min_precipitation_station = 3
  day_index, _, _ = np.unravel_index(
      min_precipitation_index - station1.size - station2.size, station3.shape
  )
  min_precipitation_day = day_index

print(
    "Najniższe opady zanotowane na stacji",
    min_precipitation_station,
    "w dniu",
    min_precipitation_day,
)

extreme_temp_days = np.any(station1[..., 0] > 30) or np.any(
    station2[..., 0] > 30
) or np.any(station3[..., 0] > 30)
extreme_humidity_days = np.any(station1[..., 1] > 45) or np.any(
    station2[..., 1] > 45
) or np.any(station3[..., 1] > 45)
extreme_precipitation_days = np.any(station1[..., 2] > 20) or np.any(
    station2[..., 2] > 20
) or np.any(station3[..., 2] > 20)

print(
    "Czy były dni z ekstremalną temperaturą?:", extreme_temp_days
)
print(
    "Czy były dni z ekstremalną wilgotnością?:", extreme_humidity_days
)
print(
    "Czy były dni z ekstremalnymi opadami?:", extreme_precipitation_days
)


correlation_station1 = np.corrcoef(station1[..., 0].flatten(), station1[..., 1].flatten())[
    0, 1
]
correlation_station2 = np.corrcoef(station2[..., 0].flatten(), station2[..., 1].flatten())[
    0, 1
]
correlation_station3 = np.corrcoef(station3[..., 0].flatten(), station3[..., 1].flatten())[
    0, 1
]

print(
    "Korelacja między temperaturą a wilgotnością na stacji 1:",
    correlation_station1,
)
print(
    "Korelacja między temperaturą a wilgotnością na stacji 2:",
    correlation_station2,
)
print(
    "Korelacja między temperaturą a wilgotnością na stacji 3:",
    correlation_station3,
)

Średnie wartości dla stacji 1:
[[25.25 66.75  7.75]
 [21.25 54.    6.5 ]
 [19.25 73.75  7.25]
 [25.5  67.25 12.25]
 [21.75 65.25  7.25]
 [21.   72.25  6.25]
 [22.   63.25  9.75]]
Średnie wartości dla stacji 2:
[[14.   53.75  4.  ]
 [21.   59.25  4.75]
 [20.75 59.    6.  ]
 [18.5  47.5   4.25]
 [14.75 50.    5.25]
 [14.5  53.25  5.25]
 [14.75 51.25  4.25]]
Średnie wartości dla stacji 3:
[[27.   75.75 14.75]
 [25.75 78.75 12.75]
 [24.75 78.   13.  ]
 [24.   76.5   9.75]
 [24.75 82.25 11.75]
 [26.   78.    9.  ]
 [23.5  71.75 10.5 ]]
Najwyższa temperatura zanotowana na stacji 1 w dniu 6
Najniższe opady zanotowane na stacji 1 w dniu 2
Czy były dni z ekstremalną temperaturą?: True
Czy były dni z ekstremalną wilgotnością?: True
Czy były dni z ekstremalnymi opadami?: False
Korelacja między temperaturą a wilgotnością na stacji 1: 0.00968208204190558
Korelacja między temperaturą a wilgotnością na stacji 2: 0.3129947139125724
Korelacja między temperaturą a wilgotnością na stacji 3: 0.21966749734